# Tensorflow with GPU

This notebook provides an introduction to computing on a [GPU](https://cloud.google.com/gpu) in Colab. In this notebook you will connect to a GPU, and then run some basic TensorFlow operations on both the CPU and a GPU, observing the speedup provided by using the GPU.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#!pip install pytorch-lightning
!pip install --quiet torchvision torch pytorch-lightning torchmetrics
#!pip install transformers==3

     |████████████████████████████████| 526 kB 4.9 MB/s 
     |████████████████████████████████| 332 kB 53.5 MB/s 
     |████████████████████████████████| 829 kB 48.6 MB/s 
     |████████████████████████████████| 133 kB 50.1 MB/s 
     |████████████████████████████████| 596 kB 44.2 MB/s 
     |████████████████████████████████| 1.1 MB 45.7 MB/s 
     |████████████████████████████████| 192 kB 52.0 MB/s 
     |████████████████████████████████| 271 kB 67.2 MB/s 
     |████████████████████████████████| 160 kB 69.6 MB/s 


In [3]:
import os
import pandas as pd
import pytorch_lightning as pl
import tensorflow as tf
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, Resize, ToTensor, Normalize
from torchvision import models
from torch.optim import AdamW

In [4]:
#CONSTANTS
ROOT = os.path.join('/content', 'drive', 'MyDrive', 'kvasir-capsule', 'labelled_images2')
#BATCH_SIZE = 4
BATCH_SIZE = 16
AVAILABLE_GPUS = min(torch.cuda.device_count(), 1)

In [5]:
#!ls -al /content/drive/MyDrive/kvasir-capsule/labelled_images/131368cc17e44240_31508*

In [6]:
#data = pd.read_csv(os.path.join(ROOT, 'metadata.csv'))
#data.head()

In [7]:
#print(data.finding_class.unique())

In [8]:
#data['finding_class'] = data['finding_class'].map({"Ampulla of Vater": 0, "Angiectasia": 1, "Blood - fresh": 2,  
#                                                   "Blood - hematin": 3, "Erosion": 4, "Erythema": 5,  
#                                                   "Foreign Body": 6, "Ileocecal valve": 7, "Lymphangiectasia": 8, 
#                                                   "Normal clean mucosa": 9, "Polyp": 10, "Pylorus": 11, 
#                                                   "Reduced Mucosal View": 12, "Ulcer": 13})

#X, y = data.filename, data.finding_class

#X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state = 0)
#print(len(X_train), len(X_valid))

In [9]:
#print(y)

In [10]:
#class KvasirImageFolder(ImageFolder):

In [11]:
#class KvasirDataset(Dataset):
#  def __init__(self, X: list, y: list, root: str, transforms: Compose = None):
#    self.X = list(X)
#    self.y = list(y)
#    self.root = root
#    self.transforms = transforms

#  def __getitem__(self, index):
#    filename = os.path.join(self.root, self.X[index])
#    label = self.y[index]
#    image = Image.open(filename)

#    if self.transforms is not None:
#      image = self.transforms(image)

#    return image, label

#  def __len__(self):  
#    return len(self.X)

In [12]:
#train_dataset1 = KvasirDataset(X_train, y_train, ROOT, transforms = Compose([
#  Resize(299),
#  ToTensor(),
#  Normalize(mean=[0.485, 0.456, 0.406],
#                                 std=[0.229, 0.224, 0.225]),                                                                                       
#]))

In [13]:
#valid_dataset1 = KvasirDataset(X_valid, y_valid, ROOT, transforms = Compose([
#  Resize(299),
#  ToTensor(),
#  Normalize(mean=[0.485, 0.456, 0.406],
#                                 std=[0.229, 0.224, 0.225]),                                                                                       
#]))
#test_dataset = KvasirDataset()

In [15]:
#"/content/drive/MyDrive/kvasir-capsule/labelled_images2/"
train_dataset = ImageFolder(os.path.join("/content/drive/MyDrive/kvasir-capsule/labelled_images2/"), transform = Compose([                                 
  Resize(299),
  ToTensor(),
  Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),                                                                                       
]))


In [16]:
print(train_dataset.class_to_idx)

{'Ampulla of vater': 0, 'Angiectasia': 1, 'Blood - fresh': 2, 'Blood - hematin': 3, 'Erosion': 4, 'Erythema': 5, 'Foreign body': 6, 'Ileocecal valve': 7, 'Lymphangiectasia': 8, 'Normal clean mucosa': 9, 'Polyp': 10, 'Pylorus': 11, 'Reduced mucosal view': 12, 'Ulcer': 13}


In [ ]:
#valid_dataset = ImageFolder(os.path.join("/content/drive/MyDrive/kvasir-capsule/labelled_images2/"), transform = Compose([                                 
#  Resize(299),
#  ToTensor(),
#  Normalize(mean=[0.485, 0.456, 0.406],
#                                 std=[0.229, 0.224, 0.225]),                                                                                       
#]))

In [17]:
len(train_dataset)

47238

In [ ]:
#valid_dataset = ImageFolder(os.path.join("/content/drive/MyDrive/kvasir-capsule/labelled_images2/"), transform = Compose([
#  Resize(299),
#  ToTensor(),
#  Normalize(mean=[0.485, 0.456, 0.406],
#                                 std=[0.229, 0.224, 0.225]),                                                                                       
#]))

In [ ]:
#valid_dataset[1]

In [18]:
train_dataset[1]

(tensor([[[-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          ...,
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179],
          [-2.1179, -2.1179, -2.1179,  ..., -2.1179, -2.1179, -2.1179]],
 
         [[-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          ...,
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357],
          [-2.0357, -2.0357, -2.0357,  ..., -2.0357, -2.0357, -2.0357]],
 
         [[-1.8044, -1.8044, -1.8044,  ..., -1.8044, -1.8044, -1.8044],
          [-1.8044, -1.8044,

In [19]:
train_dataloader = DataLoader(train_dataset, batch_size = BATCH_SIZE)
#valid_dataloader = DataLoader(valid_dataset, batch_size = BATCH_SIZE)
#valid_dataloader = DataLoader(valid_dataset, batch_size = 1)

In [20]:
class Model(pl.LightningModule):
    def __init__(self, classes: int):
      super(Model, self).__init__()

      self.classes = classes
      self._create_model()

      self.acc = Accuracy()
      self.val_acc = Accuracy()
      self.test_acc = Accuracy()
      
    def _create_model(self):
      
      #self.model = models.inception_v3(pretrained=True)
      #inputs = self.model.fc.in_features
      #self.model.fc = nn.Linear(inputs, self.classes)

      #self.model = models.mobilenet_v2(pretrained=True)
      #inputs = self.model.classifier[1].in_features
      #self.model.classifier[1] = nn.Linear(inputs, self.classes)
    
      self.model = models.densenet161(pretrained=True)
      inputs = self.model.classifier.in_features
      self.model.classifier = nn.Linear(inputs, self.classes)

    def forward(self, x):
      return self.model(x)

    def training_step(self, batch, batch_idx):
      images, targets = batch
      preds = self(images)

      loss = F.cross_entropy(preds, targets)
      self.acc(preds, targets)
      self.log('acc', self.acc, on_step=False, on_epoch=True, prog_bar=True)

      return loss
  
    #def validation_step(self, batch, batch_idx):
    #  images, targets = batch
    #  preds = self(images)
        
    #  loss = F.cross_entropy(preds, targets)
    #  self.val_acc(preds, targets)
    #  self.log('val_acc', self.val_acc, on_step=False, on_epoch=True, prog_bar=True)
    #  self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        
    #  return loss

    def configure_optimizers(self):
      return torch.optim.Adam(self.parameters(), lr=0.0001)




In [21]:
print(models.densenet161())

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(96, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(192, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(144, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (rel

In [ ]:
#print(max(data.finding_class.unique()))

In [ ]:
#print(data.columns.to_list())

In [22]:
for target_class in sorted(train_dataset.class_to_idx.keys()):
        classes = train_dataset.class_to_idx[target_class]

In [23]:
print(classes)

13


In [24]:
#classes = max(data.finding_class.unique())
model = Model(classes)
trainer = pl.Trainer(
    gpus=AVAILABLE_GPUS,
    #max_epochs=30
    max_epochs=30
)

Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /root/.cache/torch/hub/checkpoints/densenet161-8d451a50.pth


  0%|          | 0.00/110M [00:00<?, ?B/s]

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
trainer.fit(model, train_dataloader)

Missing logger folder: /content/lightning_logs

  | Name     | Type     | Params
--------------------------------------
0 | model    | DenseNet | 26.5 M
1 | acc      | Accuracy | 0     
2 | val_acc  | Accuracy | 0     
3 | test_acc | Accuracy | 0     
--------------------------------------
26.5 M    Trainable params
0         Non-trainable params
26.5 M    Total params
106.003   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

## Enabling and testing the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

Next, we'll confirm that we can connect to the GPU with tensorflow: